In [2]:
import requests
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [7]:
def get_soup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64)'}
    page = requests.get(url, headers=headers)
    return BeautifulSoup(page.content, 'html.parser')

In [5]:
def get_url(query, loc):
    url = 'https://ca.indeed.com/jobs?'
    query = 'q=' + urllib.parse.quote_plus(query)
    loc = '&l=' + urllib.parse.quote_plus(loc)
    url += query + loc
    return url


In [8]:
soup = get_soup(get_url('Carpenter', 'Vancouver, BC'))
soup

<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<script src="//d3fw5vlhllyvee.cloudfront.net/s/175fa2f/en_CA.js" type="text/javascript"></script>
<link href="//d3fw5vlhllyvee.cloudfront.net/s/b512638/jobsearch_all.css" rel="stylesheet" type="text/css"/>
<link href="http://ca.indeed.com/rss?q=Carpenter&amp;l=Vancouver%2C+BC&amp;radius=25" rel="alternate" title="Carpenter Jobs in Vancouver, BC" type="application/rss+xml"/>
<link href="/m/jobs?q=Carpenter&amp;l=Vancouver%2C+BC&amp;radius=25" media="only screen and (max-width: 640px)" rel="alternate"/>
<link href="/m/jobs?q=Carpenter&amp;l=Vancouver%2C+BC&amp;radius=25" media="handheld" rel="alternate"/>
<script type="text/javascript">

if (typeof window['closureReadyCallbacks'] == 'undefined') {
window['closureReadyCallbacks'] = [];
}

function call_when_jsall_loaded(cb) {
if (window['closureReady']) {
cb();
} else {
window['closureReadyCallbacks'].push(cb);
}
}
</scri

In [9]:
regex = re.compile('.*jobsearch-SerpJobCard unifiedRow.*')

In [10]:
cards = soup.find_all(class_=regex)
cards

[<div class="jobsearch-SerpJobCard unifiedRow row result" data-ci="45046845" data-empn="9583979371728883" data-jk="da16160cd0745d00" id="pj_da16160cd0745d00">
 <style>
 .jobcard_logo{margin:6px 0}.jobcard_logo img{width:auto;max-width:80px;max-height:30px}
 </style>
 <h2 class="title">
 <a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0AJzDcWE9tCxC00bafy-sY5Si3tTlCU8xLqcLjIdJMVfkKVNlnmzk94zhgzct2ewaBSvEHg5_Hc3a57vH1qKEbn5hoN5f_UTTzr1juVFlNKUI5gnT2rflk0uIU04CPOyG8kEDO2aucJG6I80BIn05A98U-U7cSzUPS_MAueWq5A5SjgMLqdS_qLvdiN8zeDNiNiUQzFHnYWyMmQhA-fcEqC4ryChKIJtvI4cpFYs9JToPr-FySHgkVFWDgeOfIoFXknTJioYKuM6kFMaQQpQqj-t6RixHH_h1HIKuSZtU2xZgjnB91T2D0DyJDm3tn5Awzg8ERQuw9G8OnO8vDWXduZoHI-I6qlAi99CqMSkWBA85VfCdrwJ5VYckmuePnQYvuKu0cfYtM-4aIY1ylvU_TjbenNtJnau7HHwFsWD36850DL0oOdwpJI7_QdWKO3WEwuWw-4BXqbkHXl2FDCm7-DA9NSVFQyq7N_m6k3u7aqww==&amp;p=0&amp;fvj=1&amp;vjs=3" id="sja0" onclick="setRefineByCookie([]); sjoc('sja0', 0); convCtr('SJ'); rclk(this,job

In [21]:
query = 'carpenter'
loc = 'Vancouver, BC'

for start in np.arange(0, 31, 10):
    url = get_url(query, loc) + '&start='+str(start)
    soup = get_soup(url)
    
    regex = re.compile('.*jobsearch-SerpJobCard unifiedRow.*')
    cards = soup.find_all(class_=regex)
    
    if start == 0:
        title, company, salary,  location, post_url = [], [], [], [], []

    for card in cards:
        if str(card.find(class_='title')) != 'None':
            title.append(card.find('h2', class_='title').find('a')['title'])
            company.append(card.find(class_='company').text.strip())

            if str(card.find(class_='salaryText')) != 'None':
                salary.append(card.find(class_='salaryText').text.strip())
            else:
                salary.append('None')

            location.append(card.find(class_='location accessible-contrast-color-location').text)
            post_url.append(card.find(class_='title').find('a')['href'])

In [23]:
df = pd.DataFrame(title, columns=['title'])
df['company'], df['salary'], = company, salary
df['location'], df['post_url'] = location, post_url

df

,title,company,salary,location,post_url
0,Carpenters/Carpenter Apprentice/Labourers,Heatherbrae Builders Co. Ltd.,None,"Vancouver, BC",/pagead/clk?mo=r&ad=-6NYlbfkN0AJzDcWE9tCxC00ba...
1,Framing Carpenter,Harewood Construction Ltd.,$27 - $30 an hour,"Vancouver, BC",/pagead/clk?mo=r&ad=-6NYlbfkN0A494VTyWNwcZ-3wY...
2,Apprentice Carpenter,Coastal builders,$20 - $22 an hour,"Vancouver, BC",/pagead/clk?mo=r&ad=-6NYlbfkN0AKO2dQQDYQ5BAKtF...
3,Carpenter,Centra Construction,$25 - $35 an hour,"Vancouver, BC",/pagead/clk?mo=r&ad=-6NYlbfkN0Da-VEqSSxW5mMKUC...
4,Apprentice Carpenter / Carpenter (Level 2-4),Unitech Construction Management Ltd.,None,"Richmond, BC",/pagead/clk?mo=r&ad=-6NYlbfkN0DgdpXyOwyqbxYmn-...
...,...,...,...,...,...
57,Framer Lead Hand,LUX Quality Construction,$25 - $35 an hour,"Vancouver, BC",/company/LUX-Quality-Construction/jobs/Framer-...
58,Precision Woodworker,ChopValue Manufacturing Ltd.,None,"Vancouver, BC",/company/ChopValue/jobs/Precision-Woodworker-b...
59,carpenter,Westcorp Developments Ltd.,$27.50 an hour,"Delta, BC",/rc/clk?jk=3df96a7e587b8345&fccid=6124a423a97b...
60,framer-carpenter,M.A Woodworking Ltd.,$27.50 an hour,"Surrey, BC",/rc/clk?jk=c1b5beca84c39691&fccid=ae72762707c9...


In [25]:
df.to_csv('jobs.csv', index=False, sep=';')